In [1]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import DeepLake
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# LLM
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

# Embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Create dataset fresh
texts = [
    "Napoleon Bonaparte was born on 15 August 1769.",
    "Louis XIV was born on 5 September 1638."
]

splitter = RecursiveCharacterTextSplitter()
docs = splitter.create_documents(texts)

db = DeepLake(
    dataset_path="./deeplake_dataset",
    embedding=embeddings
)

db.add_documents(docs)

# Retriever
retriever = db.as_retriever()

# Prompt
prompt = ChatPromptTemplate.from_template(
    """Answer the question using only the context below:

{context}

Question: {question}
"""
)

# RAG pipeline
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

response = rag_chain.invoke("When was Napoleon born?")

print("Final Answer:")
print(response.content)


C:\Users\Chala\OneDrive\Desktop\LangChain01\langchain_env\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.5.2) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
C:\Users\Chala\AppData\Local\Temp\ipykernel_24444\1029541057.py:28: LangChainDeprecationWarning: This class is deprecated and will be removed in a future version. You can swap to using the `DeeplakeVectorStore` implementation in `langchain-deeplake`. Please do not submit further PRs to this class.See <https://github.com/activeloopai/langchain-deeplake>
  db = DeepLake(
Creating 2 embeddings in 1 batches of size 2:: 100%|█████████████████████████████████████| 1/1 [00:00<00:00, 19.02it/s]


Dataset(path='./deeplake_dataset', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype     shape     dtype  compression
  -------    -------   -------   -------  ------- 
   text       text      (2, 1)     str     None   
 metadata     json      (2, 1)     str     None   
 embedding  embedding  (2, 384)  float32   None   
    id        text      (2, 1)     str     None   
Final Answer:
Napoleon Bonaparte was born on 15 August 1769.
